In [4]:
""" This Notebook takes the data from CBS input-output matrices to generate realistic firm to firm networks.
"""

import pandas as pd
import numpy as np

# Import CBS data for 2015
data = pd.read_excel('data/Input-Output tables 2015-2018.xlsx', sheet_name=0,
                     skiprows=1, nrows=100, usecols='B:CS', index_col=0)
data = data.drop(data.index[0]).fillna(0)

# Improve string quality
data.columns = [s.strip().lower().replace(' 1)', '') for s in data.columns]
data.index = [s.strip().lower() for s in data.index]

# Extract intermediate consumption matrix
Z = data.values[0:81, 0:81]

# Extract import and exports
imports = data.loc[['imports of goods', 'imports of services'], :'goods and services n.e.c.'].values.sum(axis=0)
exports = data[['exports of goods', 'exports of services']].values[:81].sum(axis=1)

# Extract household consumption
C = data[['final consumption expenditure of households', 
          'final consumption expenditure of npis serving households',
          'social transfers in kind']].values[:81].sum(axis=1)

# Extract government consumption
G = data[['other individual final consumption of general goevernment', 
          'collective final consumption of general gorvernment']].values[:81].sum(axis=1)

# Extract capital and inventories accumulation
capital = np.squeeze(data[['gross fixed capital formation']].values[:81])
change_inventories = np.squeeze(data[['changes in inventories']].values[:81])

# Extract other adjustments
other_demand = data[['trade and transport margins', 
                     'cif/fob-adjustment and reclassification',
                     'cif/fob-reclassification']].values[:81].sum(axis=1)

# Get total demand for checking
D = np.squeeze(data[['total']].values[:81])
assert np.allclose(D, Z.sum(axis=1) + exports + C + G + capital + change_inventories + other_demand)

# Get salaries expenditure
salaries = data.loc[['wages and salaries', 
                     'employers\' social contributions'], 
                    :'goods and services n.e.c.'].values.sum(axis=0)

# Get all duties and taxes expenditure
taxes = data.loc[['import duties', 
                  'other taxes on imports',
                  'taxes on domestic products',
                  'other taxes on production'], 
                 :'goods and services n.e.c.'].values.sum(axis=0)

# Get all subsidies
subsidies = data.loc[['import subsidies (-)', 
                      'subsidies on domestic products (-)',
                      'other subsidies on production (-)'], 
                     :'goods and services n.e.c.'].values.sum(axis=0)


# Get all other expenses
other_expenses = data.loc[['trade and transport margins', 
                           'operating surplus (gross)',
                           'cif/fob-correctie'], 
                          :'goods and services n.e.c.'].values.sum(axis=0)

# Get value added
value_added = np.squeeze(data.loc['value added at basic prices (92–96)', 
                                   :'goods and services n.e.c.'].values)

# Get total expenditure for checking
expenses = np.squeeze(data.loc['total', :'goods and services n.e.c.'].values)
assert np.allclose(expenses, Z.sum(axis=0) + imports + salaries - subsidies + taxes + other_expenses)

KeyError: "None of [Index(['import subsidies (-)', 'subsidies on domestic products (-)',\n       'other subsidies on production (-)'],\n      dtype='object')] are in the [index]"

In [5]:
data.index


Index(['crop, animal production, hunting and related activities',
       'forestry and logging', 'fishing and aquaculture',
       'extraction of crude petroleum and natural gas',
       'other mining and quarrying, except petroleum and gas',
       'mining support activities', 'manufacture of food products',
       'manufacture of beverages', 'manufacture of tobacco products',
       'manufacture of textiles, wearing apparel and leather',
       'manufacture of wood and products, except furniture',
       'manufacture of paper and paper products',
       'printing and reproduction of recorded media',
       'manufacture of coke and refined petroleum products',
       'manufacture of chemicals and chemical products',
       'manufacture of pharmaceutical products and preparations',
       'manufacture of rubber and plastic products',
       'manufacture of building materials', 'manufacture of basic metals',
       'manufacture of metal products, except machinery, equipment',
       'ma